In [1]:
from requests import get
import json
import pandas as pd

In [2]:
# hev rå data ud af tv2s og konverterede manuelt til en json
storkredse = json.load(open("raw_data/TV2/greaterConstituencies.json"))

In [53]:
# bemærkede at tv hentede svarende på den her måde
svar = {}
for storkreds in storkredse:
    print(storkreds['value'], storkreds['label'])
    res = get(f"https://election-api.services.tv2.dk/fv/fv22/candidatetest/answers/{storkreds['value']}")
    svar[storkreds['label']] = res.json()
#gem svarerne
json.dump(svar, open('raw_data/TV2/alle_svar.json', 'w'))

9000400 Bornholms Storkreds
9000600 Fyns Storkreds
9000200 Københavns Omegns Storkreds
9000100 Københavns Storkreds
9001000 Nordjyllands Storkreds
9000300 Nordsjællands Storkreds
9000500 Sjællands Storkreds
9000700 Sydjyllands Storkreds
9000900 Vestjyllands Storkreds
9000800 Østjyllands Storkreds


In [3]:
#svar = json.load(open('raw_data/TV2/alle_svar.json'))

In [4]:
# pt ignorere vi "regionale spørgsmål" fordi de i sagens natur kun er besvaret af et mindretal af kandidaterne...måske behøver jeg ikke gøre det...
tv2_spg = ["1a", "1b", "2a", "2b", "3a", "3b", "4a", "4b", "5a", "5b", "6a", "6b", "7a", "7b", "8a", "8b", "9a", "9b", "10a", "10b", "11a", "11b", "12a", "12b"]

In [5]:
# smæk det hele pænt sammen

tv2_svar = {}
tv2_kandidater = {}
for storkreds, kandidater in svar.items():
    for kandidat in kandidater:
        tv2_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'storkreds': kandidat['areaName'], 'parti': kandidat['partyName'], 'alder': kandidat['age']}
        tv2_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in tv2_spg if x in kandidat['answers']}

In [6]:
tv2_kandidater = pd.DataFrame(tv2_kandidater).T
tv2_svar = pd.DataFrame(tv2_svar).T

In [7]:
# norm mellem 0 og 1
ma = tv2_svar.max()
mi = tv2_svar.min()
tv2_svar = (tv2_svar-mi)/(ma-mi)

In [8]:
q = tv2_svar.copy()
q['parti'] = [tv2_kandidater.loc[x, 'parti'] for x in tv2_svar.index]
q['navn'] = [tv2_kandidater.loc[x, 'navn'] for x in tv2_svar.index]
q['storkreds'] = [tv2_kandidater.loc[x, 'storkreds'] for x in tv2_svar.index]
q['alder'] = [tv2_kandidater.loc[x, 'alder'] for x in tv2_svar.index]

In [9]:
# DR's data er...mere gemt...men jeg fandt en acceptabel måde at hive det ud på via scrapy
import pickle
import pandas as pd

In [10]:
objects = []
with (open("TakeTheDR/outputs.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [11]:
pd.DataFrame([[len(y) for y in x['kandidat']] for x in objects])[0].value_counts()

25    838
0     125
Name: 0, dtype: int64

In [12]:
# 838 har svaret.

In [13]:
# Vi beholder kun dem der har svaret.
objects = [x['kandidat'] for x in objects if x['kandidat'][0]]

In [14]:
sum(pd.Series([pd.DataFrame(x[0])['Answer'].sum() for x in objects])==0)
# hmm og 11 har enten svaret total nej til alt eller har ikke rigtigt svaret

11

In [15]:
objects = [x for x in objects if pd.DataFrame(x[0])['Answer'].sum() != 0]

In [16]:
len(objects)

827

In [17]:
beggetests = pd.Series([sum(q.navn == " ".join([x[1]['Firstname'], x[1]['LastName']])) for x in objects])
beggetests.mean()

0.8573155985489722

In [18]:
#ok 86% af folkenene findes i begge tests...

In [19]:
# Who the fuck tænker "det springer jeg lige over"....
# Lets find out =)

In [20]:
navne = pd.Series([" ".join([x[1]['Firstname'], x[1]['LastName']]) for x in objects])

In [21]:
navne[beggetests == 0]

8               Pavia Jacobsen
9               Martin Kjærulf
11     Chresten Heesgård Ibsen
16         Thor Clasen Jonasen
18          Rasmus Bøgh Vinter
                ...           
789        Karsten Jul Brædder
792               Rico Djernis
802       Allan Bilgaard Beier
806         Peter Zinckernagel
825                  Ida Auken
Length: 118, dtype: object

In [22]:
# Ida..tsk tsk
# Nå ..videre

In [23]:
bogstaver = json.load(open('various.json'))['bogstav_leg']

In [55]:
dr_data = []
for i, dd in enumerate(objects):
    person = {'navn': " ".join([dd[1]['Firstname'], dd[1]['LastName']]), 'parti': bogstaver[dd[1]['CurrentPartyCode']]}
    svar = (pd.DataFrame(dd[0])[['QuestionID', 'Answer']].set_index('QuestionID').T-1)/4
    svar.columns = svar.columns.astype('str')
    svar['navn'] = person['navn']
    svar['parti'] = person['parti']
    svar.index = [f'DR{i}']
    dr_data.append(svar)
dr_df = pd.concat(dr_data)

In [56]:
all_data = dr_df.merge(q, on=['navn', 'parti'])

In [57]:
# udfyld manglende svar med partiets gennemsnit
dk_spg = ['530', '531', '533', '534', '535', '537', '538', '540', '541', '543','544', '545', '546', '547', '548', '550', '551', '552', '553', '555',
       '556', '557', '559', '561', '563', '1a', '1b', '2a','2b', '3a', '3b', '4a', '4b', '5a', '5b', '6a', '6b', '7a', '7b', '8a',
       '8b', '9a', '9b', '10a', '10b', '11a', '11b', '12a', '12b']
parti_linjen = round(all_data.groupby('parti')[dk_spg].mean()*4)  # gennemsnitssvaret per parti, rundet til 1-5 (0-4) svarmulighederne

def fald_ind(x):
    x.loc[x[x.isna()].index] = parti_linjen.loc[x.parti, x[x.isna()].index]
    return x
    
all_data_final = all_data.apply(fald_ind, axis=1)

In [58]:
all_data_final.to_feather("2022_Lasse_data.feather")